In [ ]:
#@title blank template
#@markdown This notebook from [github.com/matteoferla/pyrosetta_help](https://github.com/matteoferla/pyrosetta_help).

#@markdown It can be opened in Colabs via [https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta-dimer.ipynb](https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta-dimer.ipynb)

#@markdown This notebook is intened for use with a [ColabFold output](https://github.com/sokrypton/ColabFold) from a complex of two protein.
#@markdown It loads, relaxes (=energy minises) and scores the interface of these.
#@markdown Some tricks discussed in https://blog.matteoferla.com/2021/08/tweaking-alphafold2-models-with.html


In [ ]:
#@title Installation
# ================================================================

#@markdown Modules installed pyrosetta

# ================================================================
#@markdown Installing PyRosetta with optional backup to your drive (way quicker next time!).
#@markdown Note that PyRosetta occupies some 10 GB, so you'll need to be on the 100 GB plan of Google Drive (it's one pound a month).

#@markdown NB. If `use_drive` is True, you will be prompted to give permission to
#@markdown use Google Drive —_always_ remember to check strangers code against data theft: search and look for all instances of `http`, `requests` and `post` in the code.

#@markdown ### Download PyRosetta
#@markdown The following is not the real password. However, the format is similar.
username = 'boltzmann'  #@param {type:"string"}
password = 'constant'  #@param {type:"string"}
#@markdown Are these the "normal" common credentials?
#@markdown If so their hash will be checked beforehand to check if they are correct
#@markdown (we don't want colab blocked by too many typos).
hash_comparision_required = True  #@param {type:"boolean"}
#@markdown The release to install will be the latest (from [graylab.jhu.edu/download/PyRosetta4](https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python37.ubuntu/) for latest).

#@markdown Use Google Drive for PyRosetta (way faster next time, but takes up space)
#@markdown (NB. You may be prompted to follow a link and possibly authenticate and then copy a code into a box
use_drive = True  #@param {type:"boolean"}
#@markdown Installing `rdkit` and `rdkit_to_params` allows the creation of custom topologies (params) for new ligands
install_rdkit = True  #@param {type:"boolean"}

# ================================================================

import sys
import os
import importlib
import site
from typing import *


# ## Functions

def get_shell_mode() -> str:
    """
    Muppet-proofing: are we in colab?
    """
    # get_ipython is a standard libary in ipython, but not script
    from IPython import get_ipython
    shell_name = get_ipython().__class__.__name__
    if shell_name == 'Shell':
        return 'colab'
    elif shell_name == 'ZMQInteractiveShell':
        return 'jupyter'
    elif shell_name == 'TerminalInteractiveShell':
        raise RuntimeError('This is a colabs notebook. Why are you running it in the terminal?')
    else:
        raise RuntimeError(f'This is a colabs notebook. not a {shell_name}')


def assert_password(username: str, password: str) -> None:
    """
    Verify the username and password are correct without actually knowing them.
    I worry that there may be a large number of idiots that try to guess the passwords
    thus getting Colab fail2ban jailed.
    """
    import hashlib
    hashed_username = hashlib.sha256(username.encode()).hexdigest()
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    expected_hashed_username = 'cf6f296b8145262b22721e52e2edec13ce57af8c6fc990c8ae1a4aa3e50ae40e'
    expected_hashed_password = '45066dd976d8bf0c05dc8dd4d58727945c3437e6eb361ba9870097968db7a0da'
    msg = ('The hash of the {} is not as expected: ' +
           'if your username and password combo are correct and just not the academic ones, ' +
           'set `hash_comparision_required` to False.' +
           'If you dont know the password visit the Rosetta Commons site. ' +
           'The password and username for PyRosetta are DIFFERENT than Rosetta or FoldIt. ' +
           'Please do not google for username:password for this or for anything ' +
           'as stats like number of registered users is mighty important for grants and stuff.')
    assert hashed_username == expected_hashed_username, msg.format('username')
    assert hashed_password == expected_hashed_password, msg.format('password')


def install_and_import(package_name: str,
                       pypi_name: Optional[str] = None,
                       alias_name: Optional[str] = None):
    """If the module has a different name in pypi (`pypi_name`)
    than its import name (`package_name`), specify it.

         pip install pypi_name
         import package_name as alias_name
    """
    import importlib
    # I will go to hell for this, but shmeh:
    from pip._internal.cli.main import main as pip_main
    if pypi_name is None:
        pypi_name = package_name
    if alias_name is None:
        alias_name = package_name
    try:
        importlib.import_module(package_name)
    except ImportError as error:
        if error.name != package_name:
            # these are not the droids we are looking for
            raise ImportError(f'Import of {package_name} requires module {error.name}...',
                              name=error.name)
        pip_main(['install', pypi_name])
    globals()[alias_name] = importlib.import_module(package_name)



def set_workingpath():
    # ## Use drive?
    modality = get_shell_mode()
    if modality != 'colab':
        # jupyter --> stay
        return './'
    elif use_drive:
        from google.colab import drive
        drive.mount('/content/drive')
        return '/content/drive/MyDrive'
        os.chdir(_path)
    else:  # --> stay
        return '/content'

# ## Install pyrosetta
def get_latest_release_filename(username: str, password: str) -> str:
    # assumes the system is Ubuntu
    import sys, requests, re
    from IPython.display import display, HTML
    py_version = str(sys.version_info.major) + str(sys.version_info.minor)
    url_to_latest = f'https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python{py_version}.ubuntu/latest.html'
    latest_response = requests.get(url_to_latest,
                                   auth=requests.auth.HTTPBasicAuth(username, password)
                                   )
    if latest_response.status_code == 401:
        raise ValueError('Incorrect username or password!')
    elif latest_response.status_code not in (200, 300, 301, 302, 303, 304, 305, 306, 307, 308):
        display(HTML(latest_response.text))
        raise ValueError(f'Something is wrong with the url {url_to_latest}')
    return re.search(r'[uU][rR][lL]=(.*?)["\s]', latest_response.text).group(1)

def install_pyrosetta(path:str):
    import sys, importlib, site
    # is pyrosetta installed?
    if importlib.util.find_spec('pyrosetta'):
        import pyrosetta
    # is there a Pyrosetta release file in the location?
    elif any(['PyRosetta4.Release' in filename for filename in os.listdir(path) if os.path.isdir(filename)]):
        release_folder = [filename for filename in os.listdir(path)
                          if 'PyRosetta4.Release' in filename and os.path.isdir(filename)][0]
        assert not os.system(f'pip3 install -e {_path}/{release_folder}/setup/')
    # download
    else:
        # check if hash is right.
        if hash_comparision_required:
            assert_password(username, password)
        # download tar and uncompress it on the fly
        latest_filename = get_latest_release_filename(username, password)
        assert not os.system(
            f'curl -u {username}:{password} {latest_filename} | tar -x')
        import re
        pyrosetta_folder = re.sub(r'.tar.\w+$', '', latest_filename)
        assert not os.system(f'pip3 install -e {_path}/{pyrosetta_folder}/setup/')
    # refresh:
    site.main()

import site
_path = set_workingpath()
install_pyrosetta(_path)
install_and_import(package_name='Bio', pypi_name='biopython')
if install_rdkit:
    install_and_import(package_name='rdkit', pypi_name='rdkit-pypi')
    # importing these in main makes them render properly in the notebook:
    from rdkit import Chem
    from rdkit.Chem import PandasTools
install_and_import(package_name='pyrosetta_help', pypi_name='pyrosetta-help', alias_name='ph')
install_and_import(package_name='rdkit_to_params', pypi_name='rdkit-to-params')
install_and_import('py3Dmol')
site.main()

In [ ]:
#@title Start PyRosetta
import pyrosetta
import pyrosetta_help as ph

#@markdown Do not optimise hydrogen on loading:
no_optH = False #@param {type:"boolean"}
#@markdown Ignore (True) or raise error (False) if novel residue (e.g. ligand)
ignore_unrecognized_res=False  #@param {type:"boolean"}
#@markdown Use autogenerated PDB residues are often weird (bad geometry, wrong match, protonated etc.): —best do it properly and parameterise it
load_PDB_components=False  #@param {type:"boolean"}
#@markdown Ignore all waters:
ignore_waters=False  #@param {type:"boolean"}

extra_options= ph.make_option_string(no_optH=no_optH,
                                  ex1=None,
                                  ex2=None,
                                  mute='all',
                                  ignore_unrecognized_res=ignore_unrecognized_res,
                                  load_PDB_components=load_PDB_components,
                                  ignore_waters=ignore_waters)


# capture to log
logger = ph.configure_logger()
pyrosetta.init(extra_options=extra_options)


In [ ]:
#@title Load pose
folder_name =  'prediction_38cac' #@param {type:"string"}
#@markdown This will create an object called `analyser` 
#@markdown which has the attributes
#@markdown * `scores` (pandas DataFrame),
#@markdown * three dictionaries (`original_poses`, `relaxed_poses`, `phospho_poses`)
#@markdown where the key is the rank number of the pose (value)
#@markdown * `errors` a dictionary with key = rank number and values are the matrices that 

 to be a Rosetta pose vector (without returning a clone)
analyser = ph.AF2NotebookAnalyser(folder=folder_name, load_poses=True)

In [ ]:
#@title Energy minimise

#@markdown Check to constraints the C&alpha; atoms of residues with a harmonic function with AF2 the pairwise error as sd.
#@markdown This is applied to two case:
#@markdown * all non–primary-sequence–adjecent residues with an error less than 12 &Aring; and 
#@markdown * residues in different chains that are less than 15 &Aring; apart ([Rosetta-style neighbouring distance](https://blog.matteoferla.com/2020/06/love-thy-neighbours-but-select-them.html), ~C&beta; distance)

constrained_errors=True  #@param {type:"boolean"}

#@markdown Number of FastRelax cycles

cycles=3   #@param {type:"integer"}
analyser.sidechain_relax(cycles) # prevent blowing up...
if constrained_errors:
 analyser.constrain(tolerance=2)
analyser.relax(cycles)
analyser.calculate_interface()

for i in analyser.errors:
    analyser.relaxed_poses[i].pdb_info( analyser.original_poses[i].pdb_info() )
    if i != 1:
        ph.superimpose_by_pLDDT(analyser.relaxed_poses[i], analyser.relaxed_poses[1])

In [ ]:
# show Pandas table:
analyser.scores

In [ ]:
#@title Add PTMs (optional)
#@markdown This is rather weird because:
#@markdown * PhosphoSitePlus does not have an API.
#@markdown * Colabs does not have a multiline input
#@markdown * this [way to copypaste data into a remove Jupyter Notebook](bhttps://gist.github.com/matteoferla/6de39f0057d283d270f834d487e358a9) does not work in Colabs
#@markdown So for now just copy paste the PTMs table in the next cell instead of `PASTE HERE`


In [ ]:
raw = '''PASTE HERE'''

In [ ]:
#@title Add PTMs
chain = 'A'  #@param {type:"string"}
pdb_ptms = analyser.parse_phosphosite(raw, maximum=analyser.original_poses[1].chain_end(1))
analyser.make_phosphorylated(pdb_ptms, chain, cycles)

In [ ]:
#@title Save poses and pandas dataframe

new_folder_name =  'output' #@param {type:"string"}
analyser.dump(new_folder_name)

In [ ]:
# Note that nglview does not work with Colabs but py3Dmol does.
# install py3Dmol
os.system(f'pip3 install py3Dmol')
import site
site.main()
# run
import py3Dmol
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(ph.get_pdbstr(analyser.relaxed_pose[1]),'pdb')
view.zoomTo()
view

In [ ]:
#@title Upload to Michelanglo (optional)
#@markdown [Michelanglo](https://michelanglo.sgc.ox.ac.uk/) is a website that
#@markdown allows the creation, annotation and sharing of a webpage with an interactive protein viewport.
#@markdown ([examples](https://michelanglo.sgc.ox.ac.uk/gallery)).
#@markdown The created pages are private —they have a 1 in a quintillion change to be guessed within 5 tries.

#@markdown Registered users (optional) can add interactive annotations to pages.
#@markdown Leave blank for guest:

username = ''  #@param {type:"string"}
password = ''  #@param {type:"string"}

#@markdown Choose initial model.

chosen_pose_series = 'relaxed' #@param ['original', 'relaxed', 'phospho'] {type:"string"}
chosen_rank = 1  #@param {type:"integer"}

os.system(f'pip3 install michelanglo-api')
import site
site.main()
from michelanglo_api import MikeAPI
if not username:
  mike = MikeAPI.guest_login()
else:
  mike = MikeAPI(username, password)

pose = {'original': analyser.original_poses, 
        'relaxed': analyser.relaxed_poses,
        'phospho': analyser.phospho_poses}[chosen_pose_series][chosen_rank]
page = mike.convert_pdb(pdbblock=ph.get_pdbstr(pose))
page.show_link()